## 구글 코랩 이용
https://drive.google.com/file/d/1A9guKmo1uBc7tLUji8li0dx8gB53a4Um/view?usp=sharing

## 1. 2020년 ESG 등급이 없는 기업 중, 기사가 가장 많은 기업의 토픽모델링
### 대상기업 : 두산 퓨얼셀 , LDA 사용

In [ ]:
import pandas as pd
doosan = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/doosan.csv')
doosan_df = doosan['특성추출(가중치순 상위 50개)']
doosan = [wl.split(",")  for wl in doosan_df]
doosan

In [3]:
pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pyLDAvis

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import pickle
from gensim import corpora
import gensim
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
# 1) 사전만들기
dictionary = corpora.Dictionary(doosan)

# 1-1) 사전의 필터 씌우기 -> 빈도수 2번 이상 단어만 사용
dictionary.filter_extremes(no_below=2)

# 2) doc 2 bow로 사전 속 단어 빈도수 세기
corpus = [dictionary.doc2bow(text) for text in doosan]

# 3) 하이퍼 파라미터 설정
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

# 4) 모델 적용해서 top topics들 보여주기
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -6.8582.
[([(0.065003894, '발전소'),
   (0.05588759, '부생수소'),
   (0.0558875, '한화에너지'),
   (0.047000233, '부생'),
   (0.046945173, '대산'),
   (0.046765637, '한국동서발전'),
   (0.046765633, '충남'),
   (0.03786541, '대산산업단지'),
   (0.037612334, '정인섭'),
   (0.03341573, '유수경'),
   (0.028976416, '대산그린에너지'),
   (0.028953124, '한화토탈'),
   (0.028953087, '온실가스'),
   (0.028696353, '정승일'),
   (0.02188038, '산화물'),
   (0.0218802, '질소산화물'),
   (0.020064335, '부산물'),
   (0.020064179, '인증서'),
   (0.019763863, '서산'),
   (0.019747837, '연료전지발전소')],
  -1.204991235116619),
 ([(0.036989808, '중국'),
   (0.036775745, '미국'),
   (0.03329618, '한병화'),
   (0.033295218, '이산화탄소'),
   (0.033295088, '수소연료전지'),
   (0.033295084, '수소시대'),
   (0.033294857, '유럽'),
   (0.03329464, '본격화'),
   (0.033294596, '연구원'),
   (0.033294197, '차세대'),
   (0.033294104, '상아프론테크'),
   (0.03329406, '일진다이아'),
   (0.033293728, '급등세'),
   (0.033293623, '재생에너지'),
   (0.033293575, '수소차'),
   (0.033293534, '관련주'),
   (0.03329313, '충전인프라'),


In [7]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

# 5) pyLDAvis로 시각화하기
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'lda_doosan.html')

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


### 개별기업 LDA 과정은 html파일로 제작했기 때문에, html도 추가로 첨부합니다

---
## 2. 2020년 A+ 등급을 받은 기업들의 기사 토픽 모델링
### BERTopic 사용

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_total_yes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AA기업.csv')

df_total_yes

,Unnamed: 0,특성추출(가중치순 상위 50개),label.1,score.1,종목명,emo
0,44,"BNK,김지완,임추위,정기영,BNK금융지주,부울경혁신금융,후보군,임직원,정기주주총회...",Governance,0.991241,BNK금융지주,0.999991
1,45,"BNK,계열사,임추위,이사회,김지완,5명,BNK금융지주,경영자,경쟁력",Governance,0.991369,BNK금융지주,0.999958
2,46,"ESG,부산은행,한영회계법인,지배구조,친환경,BNK,1000억,최고등급,보고서,검증...",Governance,0.989037,BNK금융지주,0.999984
3,47,"지배구조,우수기업,금융회사,구조원,BNK,김지완,BNK금융지주,거버넌스,이사회,안정...",Governance,0.992719,BNK금융지주,1.999972
4,48,"ESG,지배구조,BNK금융,만큼,경영진,BNK,이해관계자,최고등급,김지완,중요성,통...",Governance,0.992563,BNK금융지주,2.000000
...,...,...,...,...,...,...
1970,13602,"효성,이사회,계열사,지주사,지배구조,위원회,ESG,전사적,구조원,한국기업지배구조원,...",Governance,0.993260,효성화학,0.988751
1971,13603,"효성,ESG,효성화학,폴리케톤,효성그룹,효성티앤씨,사외이사,효성중공업,조현준,리젠,...",Governance,0.990327,효성화학,0.999694
1972,13604,"효성그룹,효성,지배구조,위원회,이사회,ESG,보고서,구조원,지주사,소통노력,지속가능...",Governance,0.991527,효성화학,0.999729
1973,13605,"효성,지배구조,위원회,3곳,계열사,이사회,한국,평가서,지주사,ESG,최고의사결정기구...",Governance,0.991783,효성화학,0.998962


In [ ]:
yes_li = df_total_yes['특성추출(가중치순 상위 50개)'].tolist()


In [ ]:
yes_li

In [ ]:
!pip install bertopic

In [ ]:
!pip install bertopic[visualization]

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab()) #한국어 형태소 분석을 위해 커스텀 토크나이저를 만들어야 하며 이를 Mecab으로 활용하는 것
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000) # 벡터화 시킬 수 있는 최대 개수가 3000개 

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics='auto',
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(yes_li)

In [ ]:
model.get_topic_info()['Count'].sum()

1975

In [ ]:
model.visualize_topics()

In [ ]:
#특정 기사가 어떤 토픽에 가장 해당되는지
model.visualize_distribution(probs[1900])

In [ ]:
model.visualize_hierarchy(top_n_topics=50)

In [ ]:
for i in range(0,7):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('sk', 0.06121262869697822), ('esg', 0.04068098842255657), ('구조', 0.040128088311334756), ('경영', 0.038791467262194915), ('금융', 0.038175816727027564), ('kb', 0.03708642312773126), ('그룹', 0.034681297947090496), ('지배', 0.034655831923527634), ('위원회', 0.033369972728437086), ('사회', 0.031520281419332725)]
1 번째 토픽 : [('sk', 0.11730090751314033), ('화학', 0.1165182744937995), ('친환경', 0.08442909800571478), ('윤활유', 0.07144279377310347), ('석유', 0.06856074372488503), ('해유', 0.0677128518609443), ('플라스틱', 0.0668451158402239), ('나경수', 0.05948772588038537), ('기유', 0.05608451153917103), ('자회사', 0.05596951973578054)]
2 번째 토픽 : [('포스코', 0.19000161145422348), ('ndpe', 0.17775873588820992), ('인도네시아', 0.17327552632862459), ('인터내셔널', 0.14898638770677292), ('시민', 0.14323746926469763), ('정책', 0.12680995469850218), ('오일', 0.12630060094386636), ('유기물', 0.11809709714991232), ('이탄', 0.10976251256786139), ('지역', 0.10315344797132435)]
3 번째 토픽 : [('석탄', 0.2579385863564312), ('발전소', 0.19972944069232604), ('kb',

In [ ]:
model.visualize_barchart(top_n_topics=20)

---
## 3. 2020년 A 등급을 받은 기업들의 기사 토픽 모델링
### BERTopic

In [ ]:
import pandas as pd
df_total_yes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/A기업.csv')

df_total_yes

,Unnamed: 0,특성추출(가중치순 상위 50개),label.1,score.1,종목명,emo
0,23,"BGF리테일,CU,환경부,편의점,그린스토어,사용량,리테일,임직원,류왕선,직영점,한국...",Environmental,0.979090,BGF리테일,0.999971
1,24,"친환경,류왕선,환경부,편의점,그린스토어,사용량,CU,리테일,임직원,BGF리테일,서울...",Environmental,0.977464,BGF리테일,0.990058
2,25,"환경부,CU,리테일,사용량,편의점,그린스토어,BGF리테일,현판식,류왕선,임직원,이산화탄소",Environmental,0.988793,BGF리테일,0.999947
3,27,"이건준,BGF,리테일,BGF리테일,재무제표,진천,베트남,서울,편의점,컨설팅업,발전업...",Social,0.983670,BGF리테일,1.999994
4,28,"납품업자,리테일,과징금,공정위,BGF리테일,사업자,편의점,판촉비용,대규모유통업법,갑...",Governance,0.771113,BGF리테일,0.674851
...,...,...,...,...,...,...
4470,13535,"효성,지배구조,계열사,위원회,이사회,KCGS,자문사,효성그룹,조현준,거래법,반부패,...",Governance,0.992084,효성중공업,0.999901
4471,13536,"효성,ESG,지배구조,주력사,계열사,지주사,효성화학,한국기업지배구조원,A등급,한국,...",Governance,0.992705,효성중공업,1.983139
4472,13537,"효성,ESG,효성화학,폴리케톤,효성그룹,효성티앤씨,사외이사,효성중공업,조현준,리젠,...",Governance,0.990327,효성중공업,0.999694
4473,13538,"효성,지배구조,위원회,3곳,계열사,이사회,한국,평가서,지주사,ESG,최고의사결정기구...",Governance,0.991783,효성중공업,0.998962


In [ ]:
yes_li2 = df_total_yes['특성추출(가중치순 상위 50개)'].tolist()


In [ ]:
model_A = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics='auto',
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model_A.fit_transform(yes_li2)

In [ ]:
model_A.visualize_topics()

In [ ]:
model_A.visualize_hierarchy(top_n_topics=50)

In [ ]:
for i in range(0,57):
  print(i,'번째 토픽 :', model_A.get_topic(i))

0 번째 토픽 : [('경영', 0.014591304402590865), ('cj', 0.013392967572063059), ('가능', 0.012996992631104112), ('구조', 0.012557550115182411), ('지속', 0.01247916054436494), ('사회', 0.012315172171094093), ('삼성', 0.012308452212614247), ('그룹', 0.012152834857958915), ('관계자', 0.012141088874433456), ('부회장', 0.011581479614361833)]
1 번째 토픽 : [('아시아나항공', 0.058885814508787), ('항공업', 0.058833192839424044), ('항공사', 0.0563879807651841), ('대한항공', 0.056206831665639855), ('조종사', 0.043602809833706756), ('항공', 0.034453207454415005), ('한진', 0.03185062380089955), ('아시아', 0.031143592133283753), ('산업은행', 0.03112856105040433), ('고용', 0.03075148091014432)]
2 번째 토픽 : [('삼성전자', 0.04412893502067399), ('반도체', 0.04018681812551747), ('사업', 0.02868907429164836), ('10', 0.02777863709391589), ('부문장', 0.027237765730571083), ('토론회', 0.026805631728551067), ('포스코', 0.025638201805833354), ('내년도', 0.025053544718897428), ('부회장', 0.024761565635456582), ('부장', 0.024160659842063646)]
3 번째 토픽 : [('해유', 0.11411762513238001), ('기유', 0.102156223

In [ ]:
model_A.visualize_barchart(top_n_topics=12)